# Model Comparison on Customer Analyasis

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from scipy.stats import boxcox
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_selection import RFE
%matplotlib inline

In [19]:
df = pd.read_csv('customer_data.csv')
df.head()

,Unnamed: 0,state,customer_lifetime_value,response,coverage,employmentstatus,gender,income,location_code,marital_status,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,0,Arizona,4809.0,No,Basic,Employed,M,48029,Suburban,Married,...,52,0.0,3,Corporate Auto,Corporate,Offer3,Agent,293.0,Four-Door Car,Medsize
1,2,Washington,14948.0,No,Basic,Employed,M,22139,Suburban,Single,...,31,0.0,2,Personal Auto,Personal,Offer3,Call Center,480.0,Four-Door Car,Medsize
2,6,California,5035.0,No,Basic,Employed,F,37405,Urban,Married,...,99,3.0,3,Corporate Auto,Corporate,Offer2,Branch,288.0,Four-Door Car,Medsize
3,7,California,4956.0,No,Basic,Employed,M,87197,Urban,Single,...,45,0.0,3,Personal Auto,Personal,Offer2,Branch,247.0,Two-Door Car,Medsize
4,8,California,5990.0,Yes,Premium,Employed,M,66839,Suburban,Single,...,24,0.0,1,Personal Auto,Personal,Offer2,Branch,739.0,Sports Car,Medsize


In [20]:
X = df.drop('total_claim_amount', axis=1)
y = df['total_claim_amount']

In [21]:
numerics = X.select_dtypes(np.number)
numerics.drop('Unnamed: 0', axis=1, inplace=True)
numerics.describe()

C:\Users\peter\.conda\envs\DA_Environment\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
count,5870.000000,5870.000000,5870.000000,5870.000000,5870.000000,5870.000000,5870.000000
mean,6243.174446,50637.092845,86.365588,14.972402,47.726746,0.390119,2.040204
std,3172.058923,24388.055540,21.965941,10.029939,27.900584,0.919732,0.898543
min,2120.000000,10037.000000,61.000000,0.000000,0.000000,0.000000,1.000000
25%,3746.250000,28520.000000,68.000000,6.000000,24.000000,0.000000,1.000000
50%,5452.000000,48029.000000,79.000000,14.000000,47.000000,0.000000,2.000000
75%,8208.000000,70411.500000,104.000000,23.000000,71.000000,0.000000,3.000000
max,16532.000000,99981.000000,165.000000,35.000000,99.000000,5.000000,3.000000


In [22]:
categoricals = X.select_dtypes(object)

In [24]:
dummies = pd.get_dummies(categoricals, drop_first=True)
dummies

,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,coverage_Extended,coverage_Premium,employmentstatus_Employed,employmentstatus_Medical Leave,employmentstatus_Retired,...,renew_offer_type_Offer2,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Sports Car,vehicle_class_Two-Door Car,vehicle_size_Medsize,vehicle_size_Small
0,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
1,0,0,0,1,0,0,0,1,0,0,...,0,1,0,0,1,0,0,0,1,0
2,1,0,0,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,1,0
3,1,0,0,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,1,1,0
4,1,0,0,0,1,0,1,1,0,0,...,1,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5865,0,0,0,1,0,0,1,1,0,0,...,0,0,1,0,0,0,0,1,1,0
5866,1,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
5867,0,0,0,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,1,1,0
5868,0,0,1,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,1,0


In [27]:
features = pd.concat([numerics, dummies], axis=1)
features.head()

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,state_California,state_Nevada,state_Oregon,...,renew_offer_type_Offer2,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Sports Car,vehicle_class_Two-Door Car,vehicle_size_Medsize,vehicle_size_Small
0,4809.0,48029,61,7.0,52,0.0,3,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,14948.0,22139,100,34.0,31,0.0,2,0,0,0,...,0,1,0,0,1,0,0,0,1,0
2,5035.0,37405,63,8.0,99,3.0,3,1,0,0,...,1,0,0,1,0,0,0,0,1,0
3,4956.0,87197,63,35.0,45,0.0,3,1,0,0,...,1,0,0,1,0,0,0,1,1,0
4,5990.0,66839,154,33.0,24,0.0,1,1,0,0,...,1,0,0,1,0,0,1,0,1,0


In [29]:
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.3, random_state=55)
X_train

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,state_California,state_Nevada,state_Oregon,...,renew_offer_type_Offer2,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Sports Car,vehicle_class_Two-Door Car,vehicle_size_Medsize,vehicle_size_Small
2207,13996.0,48245,121,16.0,21,3.0,2,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1804,2506.0,92056,62,21.0,1,0.0,1,0,0,1,...,1,0,0,1,0,0,0,0,1,0
3803,5380.0,93818,66,7.0,25,0.0,3,0,1,0,...,0,0,0,0,0,0,0,0,0,1
5748,7218.0,45356,90,31.0,30,0.0,3,0,0,0,...,1,0,0,0,1,0,0,0,1,0
328,5324.0,31561,68,8.0,62,0.0,3,0,0,0,...,0,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,5227.0,11896,69,29.0,57,0.0,3,1,0,0,...,0,0,0,1,0,0,0,1,0,1
968,5532.0,50648,69,27.0,50,1.0,3,0,0,0,...,0,0,0,0,1,0,0,1,1,0
4391,4929.0,52507,123,23.0,55,0.0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4762,5503.0,76839,68,12.0,85,0.0,3,1,0,0,...,0,0,0,0,1,0,0,0,1,0


In [31]:
std_scaler = StandardScaler().fit(X_train)
X_train_scaled = std_scaler.transform(X_train)
X_test_scaled = std_scaler.transform(X_test)

# Modeling

In [47]:
model_list = [LinearRegression(), Lasso(alpha=.2), Ridge(), KNeighborsRegressor(n_neighbors=10, p=2)]

In [48]:
def model_application(model_list):
    r2 = dict()
    for model in model_list:
        model.fit(X_train_scaled, y_train)
        y_pred=model.predict(X_test_scaled)
        r2[model] = r2_score(y_test, y_pred)
        print(f"{model}:\n Train: {model.score(X_train_scaled, y_train).round(4)},\n Test: {model.score(X_test_scaled, y_test).round(4)},\n Rsquared: {r2[model].round(4)} \n")

In [49]:
model_application(model_list)

LinearRegression():
 Train: 0.7869,
 Test: 0.7833,
 Rsquared: 0.7833 

Lasso(alpha=0.2):
 Train: 0.7868,
 Test: 0.7836,
 Rsquared: 0.7836 

Ridge():
 Train: 0.7869,
 Test: 0.7833,
 Rsquared: 0.7833 

KNeighborsRegressor(n_neighbors=10):
 Train: 0.6121,
 Test: 0.5293,
 Rsquared: 0.5293 



In [59]:
models = [LinearRegression(), Lasso(alpha=.2), Ridge(), KNeighborsRegressor(n_neighbors=10, p=2)]
r_squared = dict()
for i in models:
    i.fit(X_train_scaled, y_train)
    y_pred = i.predict(X_test_scaled)
    r_squared[i] = r2_score(y_test, y_pred)
    print(f'{i}\n Train: {i.score(X_train_scaled, y_train).round(3)}\n Test: {i.score(X_test_scaled, y_test).round(3)}\n R2: {r_squared[i].round(3)}\n')
    

LinearRegression()
 Train: 0.787
 Test: 0.783
 R2: 0.783

Lasso(alpha=0.2)
 Train: 0.787
 Test: 0.784
 R2: 0.784

Ridge()
 Train: 0.787
 Test: 0.783
 R2: 0.783

KNeighborsRegressor(n_neighbors=10)
 Train: 0.612
 Test: 0.529
 R2: 0.529



#  Categorical and Numerical feature selection

In [60]:
model=LinearRegression()
selector = RFE(model, n_features_to_select= 8, step = 1, verbose = 1) # Step is how many features to add or drop everytime
selector.fit(X_train_scaled, y_train)

kept_features = selector.get_support(indices = True) #returns an array of integers corresponding to nonremoved features
kept_features = list(X_train.iloc[:,kept_features].columns)

X_train = selector.transform(X_train_scaled)
X_test  = selector.transform(X_test_scaled)

X_train = pd.DataFrame(X_train, columns=kept_features)
X_test  = pd.DataFrame(X_test, columns=kept_features)


display(X_train)

Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 fea

,income,monthly_premium_auto,employmentstatus_Medical Leave,location_code_Suburban,location_code_Urban,renew_offer_type_Offer2,vehicle_size_Medsize,vehicle_size_Small
0,-0.082243,1.586071,-0.270418,0.940671,-0.542377,-0.725126,0.643028,-0.481367
1,1.711255,-1.110750,-0.270418,-1.063071,-0.542377,1.379071,0.643028,-0.481367
2,1.783386,-0.927915,-0.270418,-1.063071,1.843735,-0.725126,-1.555143,2.077415
3,-0.200510,0.169097,-0.270418,0.940671,-0.542377,1.379071,0.643028,-0.481367
4,-0.765238,-0.836497,-0.270418,0.940671,-0.542377,-0.725126,0.643028,-0.481367
...,...,...,...,...,...,...,...,...
4104,-1.570267,-0.790788,3.697972,0.940671,-0.542377,-0.725126,-1.555143,2.077415
4105,0.016129,-0.790788,-0.270418,0.940671,-0.542377,-0.725126,0.643028,-0.481367
4106,0.092232,1.677489,-0.270418,0.940671,-0.542377,-0.725126,0.643028,-0.481367
4107,1.088314,-0.836497,-0.270418,0.940671,-0.542377,-0.725126,0.643028,-0.481367


# Only numerical feature selection

In [65]:
X_train, X_test, y_train, y_test=train_test_split(numerics, y, test_size=.20, random_state=42)
std_scaler=StandardScaler().fit(X_train) 

X_train_scaled=std_scaler.transform(X_train)
X_test_scaled=std_scaler.transform(X_test)

model=LinearRegression()
selector = RFE(model, n_features_to_select= 3, step = 1, verbose = 1) # Step is how many features to add or drop everytime
selector.fit(X_train_scaled, y_train)

kept_features = selector.get_support(indices = True) #returns an array of integers corresponding to nonremoved features
kept_features = list(X_train.iloc[:,kept_features].columns)

X_train = selector.transform(X_train_scaled)
X_test  = selector.transform(X_test_scaled)

X_train = pd.DataFrame(X_train, columns=kept_features)
X_test  = pd.DataFrame(X_test, columns=kept_features)

display(X_train)

Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.


,income,monthly_premium_auto,months_since_last_claim
0,-0.086045,0.345236,1.496557
1,0.080159,0.209238,0.103232
2,-1.167396,-1.014741,-0.892000
3,1.935672,-1.105406,0.401802
4,-1.078939,-0.833411,-0.195338
...,...,...,...
4691,-0.235776,-0.969409,-0.195338
4692,0.399896,1.478550,0.998941
4693,-1.006383,-0.924076,0.302278
4694,0.876518,0.073241,-0.792477
